<div class="alert alert-block alert-info">

### Table of Contents

- [Gathering](#gathering)

- [Assessing](#assessing)

- [Cleaning](#cleaning)
    
- [Insights](#insights)
</div>

<div class="alert alert-block alert-info">

# Gathering<a id='gathering'></a>

For this project, I will gather data from three different sources.

- [First source](#first_source)

- [Second source](#second_source)

- [Third source](#third_source)

- [Wrap-up](#gathering_tldr)
    
</div>

In [1]:
# This extension formats all cells in the notebook using black
%load_ext nb_black

<IPython.core.display.Javascript object>

## The first source<a id='first_source'></a>
The `twitter-archive-enhanced.csv` which was provided by *Udacity*.

This file contains the twitter archive of the [WeRateDogs twitter account](https://twitter.com/dog_rates?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor) which is
> Your Only Source For Professional Dog Ratings

I downloaded this file manually and added it to the folder of the project under its original name.

![](./Slides/Manual_Download_Proof.png)

##### Storing downloaded data into a pandas DataFrame

In [2]:
import pandas as pd

archive_tweet_df = pd.read_csv("twitter-archive-enhanced.csv")

<IPython.core.display.Javascript object>

## The second source<a id='second_source'></a>

The `image_predictions.tsv` which is hosted on *Udacity* servers. This file contains the tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network.

##### Downloading the file programmatically using *requests* Python module.

In [3]:
import requests

url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
predictions_file_name = url.rsplit("/")[-1]

response = requests.get(url)

with open(predictions_file_name, "wb") as saved_file:
    saved_file.write(response.content)

<IPython.core.display.Javascript object>

![](./Slides/Programmatic_Download_Proof.png)

##### Storing the collected data in a pandas DataFrame

In [4]:
img_pred_df = pd.read_csv(predictions_file_name, sep="\t")

<IPython.core.display.Javascript object>

## The third source<a id='third_source'></a>

Using the `tweet_id` column in the WeRateDogs Twitter archive `twitter-archive-enhanced.csv`, I will query the Twitter API for each tweet's JSON data using Python's *Tweepy* module and store each tweet's entire set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data will be written to its own line.

##### Loading secret API keys from environment

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")
ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
ACCESS_SECRET = os.getenv("ACCESS_SECRET")

<IPython.core.display.Javascript object>

##### Querying the Twitter API through Tweepy to collect tweets' data

In [7]:
%%time
import tweepy
import json

# Authentication
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

tweet_ids = pd.read_csv("twitter-archive-enhanced.csv").tweet_id

with open("tweet-json.txt", "w") as txt_file:
    # For every tweet ID
    for tweet_id in tweet_ids:
        # Try to get the tweet information
        try:
            tweet = api.get_status(tweet_id, tweet_mode="extended")
        except tweepy.TweepError:
            continue

        # If successful, write the json string to the tweet-json.txt file and append a new line
        tweet_json_string = json.dumps(tweet._json)
        txt_file.write(f"{tweet_json_string}\n")

Rate limit reached. Sleeping for: 106
Rate limit reached. Sleeping for: 304


CPU times: user 1min 17s, sys: 2.53 s, total: 1min 19s
Wall time: 33min 22s


<IPython.core.display.Javascript object>

![](./Slides/API_Querying_proof.png)

##### Saving collected tweets data into a pandas DataFrame

In [5]:
with open("tweet-json.txt", "r") as txt_file:
    current_tweet_df = pd.read_json(txt_file, lines=True)

<IPython.core.display.Javascript object>

## Gathering Wrap-up<a id='gathering_tldr'></a>

In this section, I gathered data from 3 different sources.

- `twitter-archive-enhanced.csv` which is now loaded into **`archive_tweet_df`**
- Downloaded `image_predictions.tsv` programmatically from Udacity's servers and loaded into **`img_pred_df`**
- Queried the Twitter API through Tweepy Python module and loaded the data into **`current_tweet_df`**

Random samples of the rows from the three tables are shown below

In [6]:
archive_tweet_df.sample(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1616,685198997565345792,NaN,NaN,2016-01-07 20:39:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Alfie. That is his time machine. He's ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/685198997...,11,10,Alfie,None,None,None,None
571,801538201127157760,NaN,NaN,2016-11-23 21:29:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Wallace. He'll be your chau-fur this e...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/801538201...,12,10,Wallace,None,None,None,None


<IPython.core.display.Javascript object>

In [7]:
img_pred_df.sample(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1887,847971574464610304,https://pbs.twimg.com/media/C8SZH1EWAAAIRRF.jpg,1,coffee_mug,0.633652,False,cup,0.273392,False,toilet_tissue,0.066656,False
922,702217446468493312,https://pbs.twimg.com/media/Cb7HCMkWEAAV9zY.jpg,1,golden_retriever,0.242419,True,chow,0.226800,True,cocker_spaniel,0.194086,True


<IPython.core.display.Javascript object>

In [8]:
current_tweet_df.sample(2)

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,source,in_reply_to_status_id,...,favorited,retweeted,possibly_sensitive,possibly_sensitive_appealable,lang,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status
1215,2016-03-22 01:45:15+00:00,712092745624633345,712092745624633344,This is Steven. He's inverted af. Also very he...,False,"[0, 138]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 712092738334941184, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
1187,2016-03-31 02:09:32+00:00,715360349751484417,715360349751484416,This is Bertson. He just wants to say hi. 11/1...,False,"[0, 87]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 715360342210166784, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

<div class="alert alert-block alert-info">

# Assessing<a id='assessing'></a>

- [`archive_tweet_df`](#assessing_archive)
- [`img_pred_df`](#assessing_img_pred)
- [`current_tweet_df`](#assessing_current)
- [Wrap-up](#assessing_tldr)
</div>

## `archive_tweet_df`<a id='assessing_archive'></a>

### Visual assessment

In [9]:
archive_tweet_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


<IPython.core.display.Javascript object>

In [10]:
archive_tweet_df.tail()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None
2355,666020888022790149,NaN,NaN,2015-11-15 22:32:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a Japanese Irish Setter. Lost eye...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666020888...,8,10,None,None,None,None,None


<IPython.core.display.Javascript object>

In [11]:
archive_tweet_df.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
290,838150277551247360,8.381455e+17,21955058.0,2017-03-04 22:12:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@markhoppus 182/10,NaN,NaN,NaN,NaN,182,10,None,None,None,None,None
1603,685943807276412928,NaN,NaN,2016-01-09 21:58:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is the newly formed pupper a capella grou...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/685943807...,8,10,the,None,None,pupper,None
2069,671134062904504320,NaN,NaN,2015-11-30 01:10:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Clarence. He's a western Alkaline...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/671134062...,8,10,Clarence,None,None,None,None
119,869772420881756160,NaN,NaN,2017-05-31 04:27:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Dewey (pronounced ""covfefe""). He's hav...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/869772420...,13,10,Dewey,None,None,None,None
224,849336543269576704,NaN,NaN,2017-04-04 19:03:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",At first I thought this was a dog because of t...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/849336543...,11,10,None,None,None,None,None


<IPython.core.display.Javascript object>

### Programmatic assessment

In [12]:
archive_tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

<IPython.core.display.Javascript object>

In [13]:
archive_tweet_df.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


<IPython.core.display.Javascript object>

In [14]:
archive_tweet_df.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

<IPython.core.display.Javascript object>

In [15]:
archive_tweet_df.source.unique()

array(['<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
       '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
       '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>',
       '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'],
      dtype=object)

<IPython.core.display.Javascript object>

`source` looks like an irrelevant variable to my project. An easy decision to make after taking a look at its unique values in the code cell above.

<a id='names_cell'></a>

In [16]:
# The first letter of a valid name should be uppercase
[n for n in list(archive_tweet_df.name.unique()) if not n[0].isupper()]

['such',
 'a',
 'quite',
 'not',
 'one',
 'incredibly',
 'mad',
 'an',
 'very',
 'just',
 'my',
 'his',
 'actually',
 'getting',
 'this',
 'unacceptable',
 'all',
 'old',
 'infuriating',
 'the',
 'by',
 'officially',
 'life',
 'light',
 'space']

<IPython.core.display.Javascript object>

### Findings

**Quality Issues**

- Missing values in:
    - `in_reply_to_status_id`
    - `in_reply_to_user_id`
    - `retweeted_status_id`
    - `retweeted_status_user_id`
    - `retweeted_status_timestamp`
    - `expanded_urls`
    
    
- Irrelevant data in `source` column


- Invalid values in `rating_denominator`.

    *It is understandable that values of rating numerators are through the roof because let's face it, dogs are really cute. However, the rating denominator has to conform to some schema so we can reliably extract insights based on these ratings later. This is why I am deeming any rating denominator value other than 10 to be invalid.*


- Erroneous data type for the `timestamp` column


- "None" is used to refer to missing information in `name`, `doggo`, `pupper`, `puppo` and `floofer` columns


- Inaccurately parsed dog names in `name` such as "a", "an", "my" and "by" [Reference to relevant code cell](#names_cell)


**Tidiness Issues**

- Dog age information is distributed over 4 columns (Doggo, Pupper, Floofer, Puppo), this violates the first tidy data requirement "Each variable forms a column".

## `img_pred_df`<a id='assessing_img_pred'></a>

### Visual assessment

In [17]:
img_pred_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


<IPython.core.display.Javascript object>

In [18]:
img_pred_df.tail()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
2074,892420643555336193,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False


<IPython.core.display.Javascript object>

In [19]:
img_pred_df.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
890,699323444782047232,https://pbs.twimg.com/media/CbR-9edXIAEHJKi.jpg,1,Labrador_retriever,0.309696,True,doormat,0.303700,False,sliding_door,0.077266,False
1455,777621514455814149,https://pbs.twimg.com/media/Csqqoo5WEAAMTVW.jpg,1,chow,0.999823,True,Norwich_terrier,0.000056,True,Pomeranian,0.000028,True
1880,846874817362120707,https://pbs.twimg.com/media/C8C0JYHW0AAy-7u.jpg,2,Shetland_sheepdog,0.450539,True,papillon,0.187928,True,collie,0.140068,True
2037,884562892145688576,https://pbs.twimg.com/media/DEaZQkfXUAEC7qB.jpg,1,pug,0.546406,True,French_bulldog,0.404291,True,Brabancon_griffon,0.044002,True
1867,843856843873095681,https://pbs.twimg.com/media/C7X7Ui0XgAA3m19.jpg,1,Labrador_retriever,0.922540,True,golden_retriever,0.074358,True,Great_Pyrenees,0.002325,True


<IPython.core.display.Javascript object>

### Programmatic assessment

In [20]:
img_pred_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


<IPython.core.display.Javascript object>

In [21]:
img_pred_df.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


<IPython.core.display.Javascript object>

### Findings

This table looks much cleaner which is something I anticipated, it was created by a fellow data analyst after all. 

However, ther are minor inconveniences which must be mentioned.

**Quality Issues**
- Missing records compared to `archive_tweet_df`, this table has 2075 entries while the former one has 2356 entries.
- Records of images that ~~do not have a dog in them~~ the image prediction algorithm could not identify a dog in them i.e. records that have `p1_dog` = `p2_dog` = `p3_dog` = `False`

*No Tidiness issues that I can see*

## `current_tweet_df`<a id='assessing_current'></a>

### Visual assessment

In [22]:
current_tweet_df.head()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,source,in_reply_to_status_id,...,favorited,retweeted,possibly_sensitive,possibly_sensitive_appealable,lang,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status
0,2017-08-01 16:23:56+00:00,892420643555336193,892420643555336192,This is Phineas. He's a mystical boy. Only eve...,False,"[0, 85]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892420639486877696, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
1,2017-08-01 00:17:27+00:00,892177421306343426,892177421306343424,This is Tilly. She's just checking pup on you....,False,"[0, 138]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892177413194625024, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
2,2017-07-31 00:18:03+00:00,891815181378084864,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,False,"[0, 121]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891815175371796480, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
3,2017-07-30 15:58:51+00:00,891689557279858688,891689557279858688,This is Darla. She commenced a snooze mid meal...,False,"[0, 79]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891689552724799489, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
4,2017-07-29 16:00:24+00:00,891327558926688256,891327558926688256,This is Franklin. He would like you to stop ca...,False,"[0, 138]","{'hashtags': [{'text': 'BarkWeek', 'indices': ...","{'media': [{'id': 891327551943041024, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [23]:
current_tweet_df.tail()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,source,in_reply_to_status_id,...,favorited,retweeted,possibly_sensitive,possibly_sensitive_appealable,lang,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status
2326,2015-11-16 00:24:50+00:00,666049248165822465,666049248165822464,Here we have a 1949 1st generation vulpix. Enj...,False,"[0, 120]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 666049244999131136, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
2327,2015-11-16 00:04:52+00:00,666044226329800704,666044226329800704,This is a purebred Piers Morgan. Loves to Netf...,False,"[0, 137]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 666044217047650304, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
2328,2015-11-15 23:21:54+00:00,666033412701032449,666033412701032448,Here is a very happy pup. Big fan of well-main...,False,"[0, 130]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 666033409081393153, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
2329,2015-11-15 23:05:30+00:00,666029285002620928,666029285002620928,This is a western brown Mitsubishi terrier. Up...,False,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 666029276303482880, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
2330,2015-11-15 22:32:08+00:00,666020888022790149,666020888022790144,Here we have a Japanese Irish Setter. Lost eye...,False,"[0, 131]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 666020881337073664, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [24]:
current_tweet_df.sample(5)

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,source,in_reply_to_status_id,...,favorited,retweeted,possibly_sensitive,possibly_sensitive_appealable,lang,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status
1509,2016-01-21 02:34:07+00:00,689999384604450816,689999384604450816,This is Covach. He's trying to melt the snow. ...,False,"[0, 103]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 689999372554207232, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
2110,2015-11-27 02:08:07+00:00,670061506722140161,670061506722140160,This is Liam. He has a particular set of skill...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 670061499868684291, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
1951,2015-12-05 04:25:50+00:00,672995267319328768,672995267319328768,This is Pumpkin. He can look in two different ...,False,"[0, 120]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 672995232229781504, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
490,2016-12-24 17:18:34+00:00,812709060537683968,812709060537683968,This is Brandi and Harley. They are practicing...,False,"[0, 94]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 812709052820099072, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
2296,2015-11-17 02:00:15+00:00,666435652385423360,666435652385423360,"""Can you behave? You're ruining my wedding day...",False,"[0, 125]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 666435649399033857, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

### Programmatic assessment

In [25]:
current_tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   created_at                     2331 non-null   datetime64[ns, UTC]
 1   id                             2331 non-null   int64              
 2   id_str                         2331 non-null   int64              
 3   full_text                      2331 non-null   object             
 4   truncated                      2331 non-null   bool               
 5   display_text_range             2331 non-null   object             
 6   entities                       2331 non-null   object             
 7   extended_entities              2059 non-null   object             
 8   source                         2331 non-null   object             
 9   in_reply_to_status_id          77 non-null     float64            
 10  in_reply_to_status_id_st

<IPython.core.display.Javascript object>

In [26]:
current_tweet_df.describe()

,id,id_str,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,geo,coordinates,contributors,retweet_count,favorite_count,possibly_sensitive,possibly_sensitive_appealable,quoted_status_id,quoted_status_id_str
count,2.331000e+03,2.331000e+03,7.700000e+01,7.700000e+01,7.700000e+01,7.700000e+01,0.0,0.0,0.0,2331.000000,2331.000000,2196.0,2196.0,2.600000e+01,2.600000e+01
mean,7.419079e+17,7.419079e+17,7.440692e+17,7.440692e+17,2.040329e+16,2.040329e+16,NaN,NaN,NaN,2645.884599,7430.940798,0.0,0.0,8.113972e+17,8.113972e+17
std,6.823170e+16,6.823170e+16,7.524295e+16,7.524295e+16,1.260797e+17,1.260797e+17,NaN,NaN,NaN,4474.529669,11538.398334,0.0,0.0,6.295843e+16,6.295843e+16
min,6.660209e+17,6.660209e+17,6.658147e+17,6.658147e+17,1.185634e+07,1.185634e+07,NaN,NaN,NaN,1.000000,0.000000,0.0,0.0,6.721083e+17,6.721083e+17
25%,6.782670e+17,6.782670e+17,6.757073e+17,6.757073e+17,3.589728e+08,3.589728e+08,NaN,NaN,NaN,536.000000,1293.000000,0.0,0.0,7.761338e+17,7.761338e+17
50%,7.182469e+17,7.182469e+17,7.032559e+17,7.032559e+17,4.196984e+09,4.196984e+09,NaN,NaN,NaN,1237.000000,3228.000000,0.0,0.0,8.281173e+17,8.281173e+17
75%,7.986692e+17,7.986692e+17,8.233264e+17,8.233264e+17,4.196984e+09,4.196984e+09,NaN,NaN,NaN,3070.000000,9092.000000,0.0,0.0,8.637581e+17,8.637581e+17
max,8.924206e+17,8.924206e+17,8.862664e+17,8.862664e+17,8.405479e+17,8.405479e+17,NaN,NaN,NaN,76014.000000,153411.000000,0.0,0.0,8.860534e+17,8.860534e+17


<IPython.core.display.Javascript object>

### Findings

**Quality Issues**

- Missing records compared to `archive_tweet_df`, this table has 2331 entries while the former one has 2356 entries


- `geo`, `coordinates` and `contributors` columns entirely consist of NaN values, `place` has only one non-null value


- Missing values in:
    - `in_reply_to_status_id`, `in_reply_to_status_id_str`
    - `in_reply_to_user_id`, `in_reply_to_user_id_str`
    - `extended_entities`
    - `retweeted_status`
    - `in_reply_to_screen_name`
    - `in_reply_to_user_id_str`
    - `quoted_status_id`, `quoted_status_id_str`
    - `quoted_status_permalink`
    - `quoted_status`
    - `possibly_sensitive`, `possibly_sensitive_appealable`
    
**Tidiness Issues**

- tweets' data being scattered over the two tables `archive_tweet_df` and `current_tweet_df`  violates the third tidy data requirement "Each observational unit forms a table"

## Assessing Wrap-up<a id='assessing_tldr'></a>

### Quality Issues:

**`archive_tweet_df`**
    
1. Missing values in:
    - `in_reply_to_status_id`
    - `in_reply_to_user_id`
    - `retweeted_status_id`
    - `retweeted_status_user_id`
    - `retweeted_status_timestamp`
    - `expanded_urls`


2. Irrelevant data in `source` column


3. Invalid values in `rating_denominator`


4. Erroneous data type for the `timestamp` column


5. "None" is used to refer to missing information in `name`, `doggo`, `pupper`, `puppo` and `floofer` columns


6. Inaccurately parsed dog names in `name` such as "a", "an", "my" and "by" [Reference to relevant code cell](#names_cell)


**`img_pred_df`**

7. Records of images that the image prediction algorithm could not identify a dog in

**`current_tweet_df`**

8. Missing values in:
    - `in_reply_to_status_id`, `in_reply_to_status_id_str`
    - `in_reply_to_user_id`, `in_reply_to_user_id_str`
    - `extended_entities`
    - `retweeted_status`
    - `in_reply_to_screen_name`
    - `in_reply_to_user_id_str`
    - `quoted_status_id`, `quoted_status_id_str`
    - `quoted_status_permalink`
    - `quoted_status`
    - `possibly_sensitive`, `possibly_sensitive_appealable`
    - `geo`
    - `coordinates`
    - `contributors`
    - `place`


9. The table is cluttered with irrelevant information, `id` and `id_str` being exact copies of each other for instance. `id`, `retweet_count` and `favorite_count` columns are the only relevant columns in this table.


**Quality issues not related to single table**

10. Inconsistent number of records among the three tables:

**`archive_tweet_df`**: 2356 entries

**`img_pred_df`**: 2075 entries

**`current_tweet_df`**: 2331 entries

### Tidiness Issues:

**`archive_tweet_df`**

1. The dog age information is distributed over 4 columns (`doggo`, `pupper`, `floofer` and `puppo`), this violates the first tidy data requirement "Each variable forms a column"

**Tidiness issues not related to a single table**

2. Having tweets' data scattered over the two tables `archive_tweet_df` and `current_tweet_df`  violates the third tidy data requirement "Each observational unit forms a table"

<div class="alert alert-block alert-info">

# Cleaning<a id='cleaning'></a>
- [`archive_tweet_df`](#cleaning_archive)
- [`img_pred_df`](#cleaning_img_pred)
- [`current_tweet_df`](#cleaning_current)
- [Wrap-up](#cleaning_tldr)
</div>

Before cleaning, I am copying over the data to new dataframes to avoid making any changes to the original data.

In [27]:
# Fortunately, deep is set to True by default
archive_tweet_clean = archive_tweet_df.copy()
img_pred_clean = img_pred_df.copy()
current_tweet_clean = current_tweet_df.copy()

<IPython.core.display.Javascript object>

## `archive_tweet_df`<a id='cleaning_archive'></a>

### Quality Issues

1. Missing values in:
    - `in_reply_to_status_id`
    - `in_reply_to_user_id`
    - `retweeted_status_id`
    - `retweeted_status_user_id`
    - `retweeted_status_timestamp`
    - `expanded_urls`
    

2. Irrelevant data in `source` column

**Define**

The columns mentioned above are not really relevant to the type of analysis I intend to do, so I am going to drop these columns from `archive_tweet_df` to clear the clutter. I will use pandas `drop` function to complete this task.

**Code**

In [28]:
archive_tweet_clean.drop(
    [
        "in_reply_to_status_id",
        "in_reply_to_user_id",
        "retweeted_status_id",
        "retweeted_status_user_id",
        "retweeted_status_timestamp",
        "expanded_urls",
        "source",
    ],
    axis=1,
    inplace=True,
)

<IPython.core.display.Javascript object>

**Test**

In [29]:
archive_tweet_clean.sample(5)

,tweet_id,timestamp,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1873,675145476954566656,2015-12-11 02:49:59 +0000,What an honor. 3 dogs here. Blond one is clear...,9,10,None,None,None,None,None
232,847962785489326080,2017-04-01 00:04:17 +0000,This is Georgie. He's very shy. Only puppears ...,10,10,Georgie,None,None,None,None
1444,696713835009417216,2016-02-08 15:14:57 +0000,This is Trevith. He's a Swiss Mountain Roadwoo...,9,10,Trevith,None,None,pupper,None
313,835246439529840640,2017-02-24 21:54:03 +0000,@jonnysun @Lin_Manuel ok jomny I know you're e...,960,0,None,None,None,None,None
1035,744995568523612160,2016-06-20 20:49:19 +0000,This is Abby. She got her face stuck in a glas...,9,10,Abby,None,None,None,puppo


<IPython.core.display.Javascript object>

3. Invalid values in `rating_denominator`

**Define**

Changing every record in `rating_denominator` to 10 may look like the obvious thing to do, but having a whole column in my dataframe that stores the same value in each row means I am doing something wrong.

It is more natural to divide all values in `rating_numerator` by 10 and save the result to a column named `rating` instead and then drop the `rating_denominator` and `rating_numerator` columns.

**Code**

In [30]:
archive_tweet_clean["rating"] = archive_tweet_clean["rating_numerator"] / 10
archive_tweet_clean.drop(
    ["rating_numerator", "rating_denominator"], axis=1, inplace=True
)

<IPython.core.display.Javascript object>

**Test**

In [31]:
archive_tweet_clean.head()

,tweet_id,timestamp,text,name,doggo,floofer,pupper,puppo,rating
0,892420643555336193,2017-08-01 16:23:56 +0000,This is Phineas. He's a mystical boy. Only eve...,Phineas,None,None,None,None,1.3
1,892177421306343426,2017-08-01 00:17:27 +0000,This is Tilly. She's just checking pup on you....,Tilly,None,None,None,None,1.3
2,891815181378084864,2017-07-31 00:18:03 +0000,This is Archie. He is a rare Norwegian Pouncin...,Archie,None,None,None,None,1.2
3,891689557279858688,2017-07-30 15:58:51 +0000,This is Darla. She commenced a snooze mid meal...,Darla,None,None,None,None,1.3
4,891327558926688256,2017-07-29 16:00:24 +0000,This is Franklin. He would like you to stop ca...,Franklin,None,None,None,None,1.2


<IPython.core.display.Javascript object>

4. Erroneous data type for the `timestamp` column

**Define**

Change data type of `timestamp` column using pandas' to_datetime function

**Code**

In [32]:
archive_tweet_clean["timestamp"] = pd.to_datetime(archive_tweet_clean["timestamp"])

<IPython.core.display.Javascript object>

**Test**

In [33]:
archive_tweet_clean.timestamp.head()

0   2017-08-01 16:23:56+00:00
1   2017-08-01 00:17:27+00:00
2   2017-07-31 00:18:03+00:00
3   2017-07-30 15:58:51+00:00
4   2017-07-29 16:00:24+00:00
Name: timestamp, dtype: datetime64[ns, UTC]

<IPython.core.display.Javascript object>

5. "None" is used to refer to missing information in `name`, `doggo`, `pupper`, `puppo` and `floofer` columns

**Define**

Replace all occurrences of "None" in these columns to NaN

**Code**

In [34]:
import numpy as np

columns_to_be_amended = ["name", "doggo", "pupper", "puppo", "floofer"]

archive_tweet_clean[columns_to_be_amended] = archive_tweet_clean[
    columns_to_be_amended
].applymap(lambda x: np.nan if x == "None" else x)

<IPython.core.display.Javascript object>

**Test**

In [35]:
# Expecting no "None" values
archive_tweet_clean.doggo.value_counts()

doggo    97
Name: doggo, dtype: int64

<IPython.core.display.Javascript object>

6. Inaccurately parsed dog names in `name` such as "a", "an", "my" and "by" [Reference to relevant code cell from the assessment step](#names_cell)

**Define**

Replace all occurrences of invalid dog names with NaN while making sure only entire occurrences are replaced, actual dog names containing the letter "a" for instance should not be replaced.

**Code**

In [36]:
invalid_dog_names = [
    "such",
    "a",
    "quite",
    "not",
    "one",
    "incredibly",
    "mad",
    "an",
    "very",
    "just",
    "my",
    "his",
    "actually",
    "getting",
    "this",
    "unacceptable",
    "all",
    "old",
    "infuriating",
    "the",
    "by",
    "officially",
    "life",
    "light",
    "space",
]

archive_tweet_clean.name = archive_tweet_clean.name.apply(
    lambda x: np.nan if x in invalid_dog_names else x
)

<IPython.core.display.Javascript object>

**Test**

In [37]:
archive_tweet_clean.name.value_counts()

Charlie    12
Cooper     11
Lucy       11
Oliver     11
Tucker     10
           ..
Bert        1
Biden       1
Tino        1
Marvin      1
Clybe       1
Name: name, Length: 931, dtype: int64

<IPython.core.display.Javascript object>

In [38]:
(archive_tweet_clean.name == "Cal").sum()

1

<IPython.core.display.Javascript object>

### Tidiness Issues

1. Dog age information is distributed over 4 columns (`doggo`, `pupper`, `floofer`, `puppo`), this violates the first tidy data requirement "Each variable forms a column".

**Define**

Create a new categorical column `age` that holds information about the dog's age and drop the four aforementioned columns.

**Code**

In [40]:
dog_age_columns = ["doggo", "puppo", "pupper", "floofer"]

archive_tweet_clean["age"] = pd.Series(
    [
        np.nan if all(pd.isnull(row)) else row[pd.notna(row)][0]
        for row in archive_tweet_clean.loc[:, dog_age_columns].values
    ]
).astype("category")

archive_tweet_clean.drop(dog_age_columns, axis=1, inplace=True)

<IPython.core.display.Javascript object>

**Some notes about the code cell above**

- I am sure the solution above is not the cleanest one. I have tried using pandas' `melt` but it was not suitable for this use case. **I would appreciate it if you provided a better approach to deal with this issue in your feedback.**


- Also the indexing in `row[pd.notna(row)][0]` is here for two reasons:

1. The DataFrame.values attribute is a list, so indexing the first element means we save a string in `age` instead of a list containing one element


2. Some occurrences had more than one age categorization, how can a dog possibly be a 'floofer' and a 'puppo' at the same time? 
[Here is the link to a tweet that will explain how this happened](https://twitter.com/dog_rates/status/854010172552949760), the body of the tweet is also embedded below.


<blockquote class="twitter-tweet"><p lang="en" dir="ltr">At first I thought this was a shy <strong>doggo</strong>, but it&#39;s actually a Rare Canadian <strong>Floofer</strong> Owl. Amateurs would confuse the two. 11/10 only send dogs <a href="https://t.co/TXdT3tmuYk">pic.twitter.com/TXdT3tmuYk</a></p>&mdash; WeRateDogs® (@dog_rates) <a href="https://twitter.com/dog_rates/status/854010172552949760?ref_src=twsrc%5Etfw">April 17, 2017</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

In [41]:
archive_tweet_clean.age.value_counts()

pupper     245
doggo       97
puppo       29
floofer      9
Name: age, dtype: int64

<IPython.core.display.Javascript object>

In [42]:
archive_tweet_clean

,tweet_id,timestamp,text,name,rating,age
0,892420643555336193,2017-08-01 16:23:56+00:00,This is Phineas. He's a mystical boy. Only eve...,Phineas,1.3,NaN
1,892177421306343426,2017-08-01 00:17:27+00:00,This is Tilly. She's just checking pup on you....,Tilly,1.3,NaN
2,891815181378084864,2017-07-31 00:18:03+00:00,This is Archie. He is a rare Norwegian Pouncin...,Archie,1.2,NaN
3,891689557279858688,2017-07-30 15:58:51+00:00,This is Darla. She commenced a snooze mid meal...,Darla,1.3,NaN
4,891327558926688256,2017-07-29 16:00:24+00:00,This is Franklin. He would like you to stop ca...,Franklin,1.2,NaN
...,...,...,...,...,...,...
2351,666049248165822465,2015-11-16 00:24:50+00:00,Here we have a 1949 1st generation vulpix. Enj...,NaN,0.5,NaN
2352,666044226329800704,2015-11-16 00:04:52+00:00,This is a purebred Piers Morgan. Loves to Netf...,NaN,0.6,NaN
2353,666033412701032449,2015-11-15 23:21:54+00:00,Here is a very happy pup. Big fan of well-main...,NaN,0.9,NaN
2354,666029285002620928,2015-11-15 23:05:30+00:00,This is a western brown Mitsubishi terrier. Up...,NaN,0.7,NaN


<IPython.core.display.Javascript object>

## `img_pred_df`<a id='cleaning_img_pred'></a>

### Quality Issues

7. Records of images that ~~do not have a dog in them~~ the image prediction algorithm could not identify a dog in them i.e. records that have `p1_dog` = `p2_dog` = `p3_dog` = `False`

**Define**

Drop all records where the algorithm was not able to recognize a dog in the image

**Code**

In [43]:
img_pred_clean = img_pred_clean.drop(
    index=img_pred_clean.query(" p1_dog == p2_dog == p3_dog == False ").index
)

<IPython.core.display.Javascript object>

**Test**

In [44]:
img_pred_clean

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2069,891087950875897856,https://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,1,Chesapeake_Bay_retriever,0.425595,True,Irish_terrier,0.116317,True,Indian_elephant,0.076902,False
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True


<IPython.core.display.Javascript object>

In [45]:
img_pred_clean.query("p1_dog == p2_dog == p3_dog == False")

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


<IPython.core.display.Javascript object>

### No tidiness issues detected in `img_pred_df`

## `current_tweet_df`<a id='cleaning_current'></a>

### Quality Issues
8. Missing values in:
    - `in_reply_to_status_id`, `in_reply_to_status_id_str`
    - `in_reply_to_user_id`, `in_reply_to_user_id_str`
    - `extended_entities`
    - `retweeted_status`
    - `in_reply_to_screen_name`
    - `in_reply_to_user_id_str`
    - `quoted_status_id`, `quoted_status_id_str`
    - `quoted_status_permalink`
    - `quoted_status`
    - `possibly_sensitive`, `possibly_sensitive_appealable`
    - `geo`
    - `coordinates`
    - `contributors`
    - `place`


9. The table is cluttered with irrelevant information, `id` and `id_str` being exact copies of each other for instance. `id`, `retweet_count` and `favorite_count` columns are the only relevant columns in this table.

**Define**

Only keep the `id`, `retweet_count` and `favorite_count` columns and drop others.

**Code**

In [46]:
current_tweet_clean = current_tweet_clean.loc[
    :, ["id", "retweet_count", "favorite_count"]
].copy()

<IPython.core.display.Javascript object>

**Test**

In [47]:
current_tweet_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   id              2331 non-null   int64
 1   retweet_count   2331 non-null   int64
 2   favorite_count  2331 non-null   int64
dtypes: int64(3)
memory usage: 54.8 KB


<IPython.core.display.Javascript object>

10. Inconsistent number of records among the three tables

**Define**

1. Rename `id` column in `current_tweet_clean` to `tweet_id` so that it matches other tables
2. Set the index of the three tables to `tweet_id`
3. Use pandas `index.intersection` to get the common tweet IDs between the three tables
4. Reset the index of all three tables

**Code**

In [48]:
# 1
current_tweet_clean.rename(columns={"id": "tweet_id"}, inplace=True)

# 2
archive_tweet_clean.set_index("tweet_id", inplace=True)
img_pred_clean.set_index("tweet_id", inplace=True)
current_tweet_clean.set_index("tweet_id", inplace=True)

# 3
common_ids = archive_tweet_clean.index.intersection(
    current_tweet_clean.index
).intersection(img_pred_clean.index)

# 4
archive_tweet_clean = archive_tweet_clean.loc[common_ids, :].reset_index()
img_pred_clean = img_pred_clean.loc[common_ids, :].reset_index()
current_tweet_clean = current_tweet_clean.loc[common_ids, :].reset_index()

<IPython.core.display.Javascript object>

**Test**

In [49]:
archive_tweet_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1737 entries, 0 to 1736
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   tweet_id   1737 non-null   int64              
 1   timestamp  1737 non-null   datetime64[ns, UTC]
 2   text       1737 non-null   object             
 3   name       1220 non-null   object             
 4   rating     1737 non-null   float64            
 5   age        268 non-null    category           
dtypes: category(1), datetime64[ns, UTC](1), float64(1), int64(1), object(2)
memory usage: 69.9+ KB


<IPython.core.display.Javascript object>

In [50]:
img_pred_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1737 entries, 0 to 1736
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  1737 non-null   int64  
 1   jpg_url   1737 non-null   object 
 2   img_num   1737 non-null   int64  
 3   p1        1737 non-null   object 
 4   p1_conf   1737 non-null   float64
 5   p1_dog    1737 non-null   bool   
 6   p2        1737 non-null   object 
 7   p2_conf   1737 non-null   float64
 8   p2_dog    1737 non-null   bool   
 9   p3        1737 non-null   object 
 10  p3_conf   1737 non-null   float64
 11  p3_dog    1737 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 127.3+ KB


<IPython.core.display.Javascript object>

In [51]:
current_tweet_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1737 entries, 0 to 1736
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   tweet_id        1737 non-null   int64
 1   retweet_count   1737 non-null   int64
 2   favorite_count  1737 non-null   int64
dtypes: int64(3)
memory usage: 40.8 KB


<IPython.core.display.Javascript object>

In [52]:
# Least complicated way I found to compare between series while disregarding order and indices
assert (
    set(archive_tweet_clean.tweet_id)
    == set(img_pred_clean.tweet_id)
    == set(current_tweet_clean.tweet_id)
)

<IPython.core.display.Javascript object>

### General Tidiness Issue

2. Having tweets' data scattered over the two tables `archive_tweet_clean` and `current_tweet_clean` violates the third tidy data requirement "Each observational unit forms a table"

**Define**

Merge `archive_tweet_clean` and `current_tweet_clean` into one table as they both represent the same thing.

**Code**

In [53]:
tweets_df = pd.concat([current_tweet_clean, archive_tweet_clean], axis=1)

<IPython.core.display.Javascript object>

**Test**

In [54]:
tweets_df.head()

,tweet_id,retweet_count,favorite_count,tweet_id,timestamp,text,name,rating,age
0,892177421306343426,5585,30766,892177421306343426,2017-08-01 00:17:27+00:00,This is Tilly. She's just checking pup on you....,Tilly,1.3,NaN
1,891815181378084864,3701,23134,891815181378084864,2017-07-31 00:18:03+00:00,This is Archie. He is a rare Norwegian Pouncin...,Archie,1.2,NaN
2,891689557279858688,7711,38858,891689557279858688,2017-07-30 15:58:51+00:00,This is Darla. She commenced a snooze mid meal...,Darla,1.3,NaN
3,891327558926688256,8308,37127,891327558926688256,2017-07-29 16:00:24+00:00,This is Franklin. He would like you to stop ca...,Franklin,1.2,NaN
4,891087950875897856,2786,18722,891087950875897856,2017-07-29 00:08:17+00:00,Here we have a majestic great white breaching ...,NaN,1.3,NaN


<IPython.core.display.Javascript object>

The `tweet_id` column is duplicated in the concatenated DataFrame above which is something I will have to deal with. However, having them side by side is very reassuring as we can conclude that the concatenation was successful just by looking at them since their values match on every row.

In [55]:
# Extra reassurance by comparing both columns and summing the Trues to see if they match the length of the DataFrame
assert (tweets_df.iloc[:, 0] == tweets_df.iloc[:, 3]).sum() == tweets_df.shape[0]

<IPython.core.display.Javascript object>

In [56]:
tweets_df = tweets_df.loc[:, ~tweets_df.columns.duplicated()]

<IPython.core.display.Javascript object>

In [57]:
tweets_df.head()

,tweet_id,retweet_count,favorite_count,timestamp,text,name,rating,age
0,892177421306343426,5585,30766,2017-08-01 00:17:27+00:00,This is Tilly. She's just checking pup on you....,Tilly,1.3,NaN
1,891815181378084864,3701,23134,2017-07-31 00:18:03+00:00,This is Archie. He is a rare Norwegian Pouncin...,Archie,1.2,NaN
2,891689557279858688,7711,38858,2017-07-30 15:58:51+00:00,This is Darla. She commenced a snooze mid meal...,Darla,1.3,NaN
3,891327558926688256,8308,37127,2017-07-29 16:00:24+00:00,This is Franklin. He would like you to stop ca...,Franklin,1.2,NaN
4,891087950875897856,2786,18722,2017-07-29 00:08:17+00:00,Here we have a majestic great white breaching ...,NaN,1.3,NaN


<IPython.core.display.Javascript object>

## Cleaning Wrap-up<a id='cleaning_tldr'></a>

After this exhaustive cleaning process, we now have two clean tables, namely, `tweets_df` and `img_pred_clean`.

`tweets_df` holds information about the tweets and `img_pred_clean` holds information about the images in these tweets. Both have the same number of records and contain only relevant information that we can proceed with to the insights' step.

In [58]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1737 entries, 0 to 1736
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   tweet_id        1737 non-null   int64              
 1   retweet_count   1737 non-null   int64              
 2   favorite_count  1737 non-null   int64              
 3   timestamp       1737 non-null   datetime64[ns, UTC]
 4   text            1737 non-null   object             
 5   name            1220 non-null   object             
 6   rating          1737 non-null   float64            
 7   age             268 non-null    category           
dtypes: category(1), datetime64[ns, UTC](1), float64(1), int64(3), object(2)
memory usage: 97.0+ KB


<IPython.core.display.Javascript object>

In [59]:
img_pred_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1737 entries, 0 to 1736
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  1737 non-null   int64  
 1   jpg_url   1737 non-null   object 
 2   img_num   1737 non-null   int64  
 3   p1        1737 non-null   object 
 4   p1_conf   1737 non-null   float64
 5   p1_dog    1737 non-null   bool   
 6   p2        1737 non-null   object 
 7   p2_conf   1737 non-null   float64
 8   p2_dog    1737 non-null   bool   
 9   p3        1737 non-null   object 
 10  p3_conf   1737 non-null   float64
 11  p3_dog    1737 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 127.3+ KB


<IPython.core.display.Javascript object>

# Insights<a id='insights'></a>